<a href="https://colab.research.google.com/github/khhaliil/Crab_age_prediction/blob/main/Model_1_(4_01_MSE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import gc
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
#from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
import xgboost as xgb
#import optuna

import warnings
from sklearn.model_selection import train_test_split

warnings.simplefilter("ignore")

In [14]:
train = pd.read_csv('/content/Train.csv')
test= pd.read_csv('/content/Test.csv')
train.head()
test.head()

,Gender,Measures,Weights
0,I,"(0.775, 0.5875, 0.175)","(1.7860185, 1.14815475, 1.2757275)"
1,M,"(1.1625, 0.8875, 0.3)","(7.72523875, 2.7499015, 3.95475525)"
2,F,"(1.0875, 0.875, 0.3)","(5.443104, 2.83495, 3.685435)"
3,I,"(1.35, 0.9875, 0.3375)","(7.66853975, 4.3941725, 5.443104)"
4,I,"(0.825, 0.6, 0.1875)","(2.11203775, 0.9355335, 1.360776)"


In [15]:
def LightGBM(X, y, test_data, params):
    kf = list(KFold(n_splits=10, shuffle=True, random_state=100).split(X, y))
    preds, models = [], []
    oof = np.zeros(len(X))
    imp = pd.DataFrame()

    for nfold in np.arange(10):
        print("-"*30, "fold:", nfold, "-"*30)

        # set train/valid data
        idx_tr, idx_va = kf[nfold][0], kf[nfold][1]
        x_tr, y_tr = X.loc[idx_tr, :], y.loc[idx_tr]
        x_va, y_va = X.loc[idx_va, :], y.loc[idx_va]

        # training
        model = lgb.LGBMRegressor(**params)
        model.fit(x_tr, y_tr,
                eval_set=[(x_tr, y_tr), (x_va, y_va)],
                early_stopping_rounds=300,
                verbose=500,
        )
        models.append(model)

        # validation
        pred_va = model.predict(x_va)
        oof[idx_va] = pred_va
        print("MAE(valid)", nfold, ":", "{:.4f}".format(mean_absolute_error(y_va, pred_va)))

        # prediction
        pred_test = model.predict(test_data)
        preds.append(pred_test)

        # importance
        _imp = pd.DataFrame({"features": X.columns, "importance": model.feature_importances_, "nfold": nfold})
        imp = pd.concat([imp, _imp], axis=0, ignore_index=True)

    imp = imp.groupby("features")["importance"].agg(["mean", "std"])
    imp.columns = ["importance", "importance_std"]
    imp["importance_cov"] = imp["importance_std"] / imp["importance"]
    imp = imp.reset_index(drop=False)
    display(imp.sort_values("importance", ascending=False, ignore_index=True))

    return preds, models, oof, imp


In [16]:
def Splitter(Data):
    # Splitting Measures into individual components
    Data[['Length', 'Diameter', 'Height']] = Data['Measures'].str.extract(r'\(([^,]+),([^,]+),([^)]+)\)').astype(float)

    # Splitting Weights into individual components
    Data[['Shell', 'Shucked', 'Viscera']] = Data['Weights'].str.extract(r'\(([^,]+),([^,]+),([^)]+)\)').astype(float)


    Data['Adult'] = Data['Gender'].apply(lambda x: 1 if x in ['M', 'F'] else 0)
    Data.drop('Gender', axis=1, inplace=True)
    Data=Data.drop(['Weights','Measures'],axis=1)
    return Data

train=Splitter(train)
test= Splitter(test)
X = train.drop('Age', axis=1)
y = train['Age']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=42)
print(X.head())


   Length  Diameter  Height      Shell    Shucked    Viscera  Adult
0  1.6250    1.3000  0.4750  14.713390   8.674947  12.658052      1
1  1.5875    1.2250  0.4375  17.661738   7.668540  11.198052      1
2  1.6750    1.3000  0.4875  21.488921  10.588538  14.330672      1
3  1.7375    1.4500  0.5000  19.135912  13.551061  15.011060      1
4  1.0000    0.7875  0.2250   4.280774   1.927766   2.267960      0


In [17]:
# preds_lgb, models_lgb, oof_lgb, imp_lgb = LightGBM(x_train, y_train, x_test, params)

# # MAE for LightGBM
# oof_lgb_round = np.zeros(len(oof_lgb), dtype=int)
# for i in range(len(oof_lgb)):
#     oof_lgb_round[i] = int((oof_lgb[i] * 2 + 1) // 2)

# print("MAE(int):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb_round)))
# print("MAE(float):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb)))

# # visualization of predictions by test-data
# mean_preds_lgb = np.mean(preds_lgb, axis=0)
# mean_preds_lgb_round = np.zeros(len(mean_preds_lgb), dtype=int)
# for i in range(len(mean_preds_lgb_round)):
#     mean_preds_lgb_round[i] = int((mean_preds_lgb[i] * 2 + 1) // 2)

In [18]:
from itertools import tee
import lightgbm as lgb


Test = test.copy()
lgb_model = lgb.LGBMRegressor(
    categorical_feature= [0],
    task = 'predict',
    application = 'regression',
    objective = 'mae',
    boosting_type="gbdt",
    num_iterations = 2600,
    learning_rate = 0.09,
    num_leaves=9,
    tree_learner='feature',
    max_depth =12,
    min_data_in_leaf=10,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='mae',
    feature_fraction = 0.8,
    random_state=42)

lgb_model.fit(X,y)

y_pred = lgb_model.predict(X)
mse = mean_squared_error(y, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

y_Testfinal = lgb_model.predict(Test)
final = pd.DataFrame(y_Testfinal, columns=['Age'])

final.reset_index(inplace=True)
final.rename(columns={'index': 'ID'}, inplace=True)
csv_file_path = 'Solution.csv'
final.to_csv(csv_file_path, index=False)

lgb_model.predict(X)


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, 

array([15.5581229 , 11.25281919, 11.14566694, ...,  8.81560294,
        6.09643307,  9.93859194])

In [19]:
yTest = lgb_model.predict(Test)
predictions = pd.DataFrame(test.copy())
predictions["Age"] = yTest[:]
predictions.Age = predictions.Age.apply(lambda x: round(x))
predictions.to_csv("m.csv", index= False)





[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
